In [9]:
# This module reads the output from the MapReduce in order to generate statistics for a user

import numpy as np
from csv import reader
import requests

username = input()
print()

processed_data = open("output.txt")

found = False
for i in processed_data:
    user = i[1: i.index("\"", 1)]
    if user == username:
        url = "https://lichess.org/api/user/{username}/rating-history"
        PARAMS = {'username': username}
        r = requests.get(url = url.format(**PARAMS))

        data = r.json()

        filtered_data = []

        for elem in data:

            points = elem['points']
            if(len(points)>0):
                filtered_data.append({'name': elem['name'], 'rating': points[-1][3]})
            else:
                filtered_data.append({'name': elem['name'], 'rating': 'N/A'})

        print('Current ELOs:')
        for type_elo in filtered_data:
            if type_elo['rating'] != 'N/A':
                print(f'\t- {type_elo["name"]}: {type_elo["rating"]}')
        print()
        
        found = True
        data = list(reader([i]))[0][1 :]
        
        white_data = [int(x) for x in data[data.index('WHITE') + 1 : data.index('WHITE') + 3]]
        black_data = [int(x) for x in data[data.index('BLACK') + 1 : data.index('BLACK') + 3]]
        opening_data = [int(x) if index % 3 != 0 else x 
                        for index, x in enumerate(data[data.index('OPEN') + 1 : data.index('PGN')])]
        pgn_data_raw = [int(x) if index % 2 != 0 else x
                        for index, x in enumerate(data[data.index('PGN') + 1 :])]
        pgn_data = [pgn_data_raw[i : i + 2] for i in range(0, len(pgn_data_raw), 2)]
        
        print(f'Win rate as white: {100 * white_data[0] / white_data[1]:.2f}%')
        print(f'Win rate as black: {100 * black_data[0] / black_data[1]:.2f}%')
        
        opening_stats = [opening_data[i : i + 3] for i in range(0, len(opening_data), 3)]
        opening_stats.sort(key=lambda ar: (ar[1] / ar[2]), reverse=True)
        opening_stats.sort(key=lambda ar: ar[2], reverse=True)
        
        basic_opening_info = {}
        for raw_opening, won, total in opening_stats:
            opening = raw_opening if ':' not in raw_opening else raw_opening[: raw_opening.index(':')]
            
            if opening not in basic_opening_info:
                basic_opening_info[opening] = np.array([won, total])
            else:
                basic_opening_info[opening] += np.array([won, total])
                
        opening_info = [[key, basic_opening_info[key][0], basic_opening_info[key][1]] for key in basic_opening_info]
        opening_info.sort(key=lambda ar: (ar[1] / ar[2]), reverse=True)
        opening_info.sort(key=lambda ar: ar[2], reverse=True)
        
        print()
        print('Opening Statistics (Basic):')
        for opening, won, total in opening_info:
            print('\t-', opening, '-', f'{100 * won / total:.2f}% win rate ({total} games played)')
        
        print()
        print('Opening Statistics (Advanced):')
        for opening, won, total in opening_stats:
            print('\t-', opening, '-', f'{100 * won / total:.2f}% win rate ({total} games played)')
            
        print()
        print('PGNs of games lost can be found in lost.pgn')
        with open('lost.pgn', 'w') as lost:
            for pgn, result in pgn_data:
                print('[WhiteElo "1000"]', file=lost)
                print('[BlackElo "1000"]', file=lost)
                
                result_string = '1-0' if result == 1 else '0-1'
                
                print(f'[Result "{result_string}"]', file=lost)
                print(file=lost)
                print(pgn, file=lost)
                print(file=lost)

if not found:
    print("User not found")
        
processed_data.close()

EricRosen

Current ELOs:
	- Bullet: 2759
	- Blitz: 2594
	- Rapid: 2659
	- Classical: 2334
	- Correspondence: 1741
	- Chess960: 2224
	- King of the Hill: 2212
	- Three-check: 2129
	- Antichess: 1869
	- Atomic: 2022
	- Horde: 1981
	- Racing Kings: 1901
	- Crazyhouse: 2343
	- Puzzles: 2575
	- UltraBullet: 2266

Win rate as white: 63.64%
Win rate as black: 45.45%

Opening Statistics (Basic):
	- Indian Defense - 63.64% win rate (11 games played)
	- Queen's Pawn Game - 57.14% win rate (7 games played)
	- Russian Game - 66.67% win rate (3 games played)
	- English Opening - 66.67% win rate (3 games played)
	- Modern Defense - 66.67% win rate (3 games played)
	- Queen's Gambit Declined - 33.33% win rate (3 games played)
	- Pirc Defense - 100.00% win rate (2 games played)
	- Philidor Defense - 0.00% win rate (2 games played)
	- Bishop's Opening - 100.00% win rate (1 games played)
	- Catalan Opening - 100.00% win rate (1 games played)
	- Four Knights Game - 100.00% win rate (1 games played)
	- Be